In [ ]:
import os
import cv2
from PIL import Image

input_dir = "/content/drive/MyDrive/Colab Notebooks/24-2 MODELING/GOAE/example/raw_image"
output_dir = "/content/drive/MyDrive/Colab Notebooks/24-2 MODELING/GOAE/example/real_person"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Task 1: Cropping + Extension Changing
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

for image_file in os.listdir(input_dir):
    if image_file.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(input_dir, image_file)
        img = cv2.imread(img_path)

        faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        (x, y, w, h) = faces[0]
        extended_y = y
        extended_h = int(h * 1.3)

        # 1.3 denotes the additional height to include neck part
        # (natural 3D rendering needs just more than face part)

        extended_h = min(extended_h, img.shape[0] - extended_y)
        face_with_neck = img[extended_y:extended_y+extended_h, x:x+w]
        resized_img = cv2.resize(face_with_neck, (512, 512))

        output_path = os.path.join(output_dir, os.path.splitext(image_file)[0] + ".jpg")
        cv2.imwrite(output_path, resized_img)

In [ ]:
# Task 2: Upscaling (For FADING output)
from cv2 import dnn_superres

input_dir = "/content/drive/MyDrive/Colab Notebooks/24-2 MODELING/GOAE/example/real_person"
output_dir = "/content/drive/MyDrive/Colab Notebooks/24-2 MODELING/GOAE/example/upscaled"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

sr = dnn_superres.DnnSuperResImpl_create()

model_path = "/content/drive/MyDrive/Colab Notebooks/24-2 MODELING/GOAE/models/ESPCN_x2.pb"
sr.readModel(model_path)
sr.setModel("espcn", 2)  # Scale Defining: Generally x2

for image_file in os.listdir(input_dir):
    if image_file.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(input_dir, image_file)
        img = cv2.imread(img_path)

        result = sr.upsample(img)
        result = cv2.resize(result, (512, 512))

        output_path = os.path.join(output_dir, os.path.splitext(image_file)[0] + ".jpg")
        cv2.imwrite(output_path, result)